## 5.1 Creating test dataframe to test Logreg model

In [47]:
# imports
import requests
import re
import string
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
from nltk import pos_tag
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import sklearn.feature_extraction.text 
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, mean_squared_error, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


# this setting widens how many characters pandas will display in a column:
pd.options.display.max_colwidth = 400

# Import CountVectorizer and TFIDFVectorizer from feature_extraction.text.
import sklearn.feature_extraction.text 
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix, confusion_matrix, mean_squared_error, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, plot_roc_curve
!pip install psaw

In [3]:
#Instantiating API
from psaw import PushshiftAPI
api = PushshiftAPI()

In [4]:
most_recent_subs = api.search_submissions(limit=50)
results = list(most_recent_subs)

/opt/anaconda3/lib/python3.8/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [5]:
type(results[0])

psaw.PushshiftAPI.submission

In [6]:
#check different attributes for this particular submission
results[0].d_

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'Ty3hirty',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_fp9s0jmp',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1643170690,
 'domain': 'open.spotify.com',
 'full_link': 'https://www.reddit.com/r/hiphopheads/comments/scx31w/otf_chicago_playlist_durk_von/',
 'gildings': {},
 'id': 'scx31w',
 'is_created_from_ads_ui': False,
 'is_crosspostable': False,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': False,
 'is_self': False,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media': {'oembed': {'description': 'Listen on Spotify: contac

In [7]:
import datetime
posted_after = int(datetime.datetime(2010, 1, 1).timestamp())
posted_before = int(datetime.datetime(2010, 12, 31).timestamp()) #1 year's worth in 2010


In [8]:
query = api.search_submissions(subreddit='cats', after=posted_after, before=posted_before, limit=10000)

In [9]:
submissions = list()
for element in query:
    submissions.append(element.d_)
print(len(submissions))

1589


In [10]:
testdf_cats = pd.DataFrame(submissions)

In [11]:
testdf_cats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1589 entries, 0 to 1588
Data columns (total 58 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   author                         1589 non-null   object 
 1   author_flair_background_color  499 non-null    object 
 2   author_flair_css_class         33 non-null     object 
 3   author_flair_text              33 non-null     object 
 4   author_flair_text_color        499 non-null    object 
 5   brand_safe                     1589 non-null   bool   
 6   contest_mode                   1589 non-null   bool   
 7   created_utc                    1589 non-null   int64  
 8   domain                         1589 non-null   object 
 9   full_link                      1589 non-null   object 
 10  gilded                         1589 non-null   int64  
 11  id                             1589 non-null   object 
 12  is_crosspostable               1589 non-null   b

In [13]:
testdf_cats=testdf_cats[['subreddit', 'title']]

In [15]:
testdf_cats.head() #double checking if df contains subreddit and title

,subreddit,title
0,cats,Holidays Spell Potential Danger for Cats - Thi...
1,cats,This is Mrs Charles Bronson. She takes no shit...
2,cats,My Dad's cat smells like a huge butthole. I ca...
3,cats,My cat is sick. Going to take her to the vet t...
4,cats,Awww yeeah


In [25]:
testdf_cats.to_csv('../project_3/CSV files/testdf_cats.csv', sep=';', header=True, index=False, columns=[
    'subreddit', 'title'])

In [17]:
query2 = api.search_submissions(subreddit='dogs', after=posted_after, before=posted_before, limit=10000)

In [18]:
submissions2 = list()
for element in query2:
    submissions2.append(element.d_)
print(len(submissions2))

/opt/anaconda3/lib/python3.8/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


1461


In [20]:
testdf_dogs = pd.DataFrame(submissions2)

In [21]:
testdf_dogs=testdf_dogs[['subreddit', 'title']]

In [23]:
testdf_dogs.head()

,subreddit,title
0,dogs,Farm dog
1,dogs,Mystery of why dog got stuck in wall
2,dogs,Aggressive AKK. How do I fix this?
3,dogs,An enjoyable mystery about dogs
4,dogs,"GSD allergic to something, need help figuring ..."


In [26]:
testdf_dogs.to_csv('../project_3/CSV files/testdf_dogs.csv', sep=';', header=True, index=False, columns=[
    'subreddit', 'title'])

In [29]:
dogstest = pd.read_csv("../project_3/CSV files/testdf_dogs.csv",sep=';')

In [31]:
catstest = pd.read_csv("../project_3/CSV files/testdf_cats.csv",sep=';')

In [32]:
catstest

,subreddit,title
0,cats,Holidays Spell Potential Danger for Cats - Thi...
1,cats,This is Mrs Charles Bronson. She takes no shit...
2,cats,My Dad's cat smells like a huge butthole. I ca...
3,cats,My cat is sick. Going to take her to the vet t...
4,cats,Awww yeeah
...,...,...
1584,cats,Cat vs HC Helicopter. An Exotic Short Hair Cat...
1585,cats,"Cat From Hell. Funny, or stupid animal abuse?"
1586,cats,Neelix loves the Hefeweizen
1587,cats,Neelix loves the Hefeweizen


In [33]:
frames = [dogstest,catstest]
testdf = pd.concat(frames)

In [34]:
testdf

,subreddit,title
0,dogs,Farm dog
1,dogs,Mystery of why dog got stuck in wall
2,dogs,Aggressive AKK. How do I fix this?
3,dogs,An enjoyable mystery about dogs
4,dogs,"GSD allergic to something, need help figuring ..."
...,...,...
1584,cats,Cat vs HC Helicopter. An Exotic Short Hair Cat...
1585,cats,"Cat From Hell. Funny, or stupid animal abuse?"
1586,cats,Neelix loves the Hefeweizen
1587,cats,Neelix loves the Hefeweizen


## 5.2 Preprocessing/Cleaning testdf

In [38]:
def remove_URL(sample):
    
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

testdf['clean_title'] = testdf['title'].apply(remove_URL)

In [39]:
testdf["clean_title"].replace('\d', '', regex=True, inplace=True)
testdf[["title","clean_title"]] # numbers are gone! 

,title,clean_title
0,Farm dog,Farm dog
1,Mystery of why dog got stuck in wall,Mystery of why dog got stuck in wall
2,Aggressive AKK. How do I fix this?,Aggressive AKK. How do I fix this?
3,An enjoyable mystery about dogs,An enjoyable mystery about dogs
4,"GSD allergic to something, need help figuring out what it is.","GSD allergic to something, need help figuring out what it is."
...,...,...
1584,Cat vs HC Helicopter. An Exotic Short Hair Cat battles a Air Hogs Havoc Helicopter. A Classic from last year.,Cat vs HC Helicopter. An Exotic Short Hair Cat battles a Air Hogs Havoc Helicopter. A Classic from last year.
1585,"Cat From Hell. Funny, or stupid animal abuse?","Cat From Hell. Funny, or stupid animal abuse?"
1586,Neelix loves the Hefeweizen,Neelix loves the Hefeweizen
1587,Neelix loves the Hefeweizen,Neelix loves the Hefeweizen


In [40]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
testdf["clean_title"] = testdf["clean_title"].apply(lambda s: deEmojify(s))

In [41]:
def text_process(mess): 
    """
    1. remove punc
    """
    nopunc=[char for char in mess if char not in string.punctuation]
    nopunc=''.join(nopunc)
    return nopunc

In [42]:
testdf["clean_title"] = testdf["clean_title"].apply(text_process)
testdf[["title","clean_title"]]

,title,clean_title
0,Farm dog,Farm dog
1,Mystery of why dog got stuck in wall,Mystery of why dog got stuck in wall
2,Aggressive AKK. How do I fix this?,Aggressive AKK How do I fix this
3,An enjoyable mystery about dogs,An enjoyable mystery about dogs
4,"GSD allergic to something, need help figuring out what it is.",GSD allergic to something need help figuring out what it is
...,...,...
1584,Cat vs HC Helicopter. An Exotic Short Hair Cat battles a Air Hogs Havoc Helicopter. A Classic from last year.,Cat vs HC Helicopter An Exotic Short Hair Cat battles a Air Hogs Havoc Helicopter A Classic from last year
1585,"Cat From Hell. Funny, or stupid animal abuse?",Cat From Hell Funny or stupid animal abuse
1586,Neelix loves the Hefeweizen,Neelix loves the Hefeweizen
1587,Neelix loves the Hefeweizen,Neelix loves the Hefeweizen


In [43]:
stop = set(stopwords.words('english'))
print(stop)

{'was', 'he', "mustn't", 'the', "didn't", 'from', 'as', 'against', 'ain', 'these', 'had', 'below', 'were', 'those', 'being', 'if', 'you', 'up', 'because', 'any', 'further', 'an', 'yourself', 's', 'we', 'did', 'under', 'so', 'y', 'what', 'between', 'that', 'of', 'it', 'at', 'weren', 'm', "hasn't", 'do', 'after', 'doesn', 'on', 'shan', 'her', 'or', 'yourselves', 'while', "aren't", 'theirs', "shan't", "weren't", "mightn't", 'haven', 'ours', 'about', 'few', 'are', 'hadn', "couldn't", 'then', 'herself', 'is', "isn't", 've', 'they', 'does', 'isn', 'aren', 'through', 'its', 'them', "hadn't", "that'll", 'whom', "you've", "you'll", 'your', 'who', 'a', 'both', 't', 'should', "won't", 're', 'our', 'out', 'such', 'by', "doesn't", 'd', 'their', 'this', 'him', 'than', 'just', 'into', 'will', 'why', "don't", 'have', "wasn't", 'needn', 'been', 'i', 'during', 'here', "needn't", 'until', 'too', "wouldn't", 'hers', 'with', 'having', 'which', 'himself', 'how', 'ma', 'mightn', 'again', 'and', 'no', 'be', '

In [44]:
def rem_en(input_txt):
    words = input_txt.lower().split()
    noise_free_words = [word for word in words if word not in stop] 
    noise_free_text = " ".join(noise_free_words) 
    return noise_free_text

In [45]:
testdf["clean_title"] = testdf["clean_title"].apply(lambda s: rem_en(s))
testdf[["title","clean_title"]]

,title,clean_title
0,Farm dog,farm dog
1,Mystery of why dog got stuck in wall,mystery dog got stuck wall
2,Aggressive AKK. How do I fix this?,aggressive akk fix
3,An enjoyable mystery about dogs,enjoyable mystery dogs
4,"GSD allergic to something, need help figuring out what it is.",gsd allergic something need help figuring
...,...,...
1584,Cat vs HC Helicopter. An Exotic Short Hair Cat battles a Air Hogs Havoc Helicopter. A Classic from last year.,cat vs hc helicopter exotic short hair cat battles air hogs havoc helicopter classic last year
1585,"Cat From Hell. Funny, or stupid animal abuse?",cat hell funny stupid animal abuse
1586,Neelix loves the Hefeweizen,neelix loves hefeweizen
1587,Neelix loves the Hefeweizen,neelix loves hefeweizen


In [48]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [49]:
def lemma_words(sentence): 
    doc = nlp(sentence)
    sentence_output = " ".join([token.lemma_ for token in doc])
    return sentence_output

In [50]:
testdf["clean_title"] = testdf["clean_title"].apply(lemma_words)
testdf[["title","clean_title"]]

,title,clean_title
0,Farm dog,farm dog
1,Mystery of why dog got stuck in wall,mystery dog get stuck wall
2,Aggressive AKK. How do I fix this?,aggressive akk fix
3,An enjoyable mystery about dogs,enjoyable mystery dog
4,"GSD allergic to something, need help figuring out what it is.",gsd allergic something need help figure
...,...,...
1584,Cat vs HC Helicopter. An Exotic Short Hair Cat battles a Air Hogs Havoc Helicopter. A Classic from last year.,cat vs hc helicopter exotic short hair cat battle air hogs havoc helicopter classic last year
1585,"Cat From Hell. Funny, or stupid animal abuse?",cat hell funny stupid animal abuse
1586,Neelix loves the Hefeweizen,neelix love hefeweizen
1587,Neelix loves the Hefeweizen,neelix love hefeweizen


In [51]:
ps = PorterStemmer()

In [52]:
def stem_words(text):
    text=text.lower() 
    words=re.split("\\s+",text)
    stemmed_words=[ps.stem(word=word) for word in words]
    return ' '.join(stemmed_words)

In [53]:
testdf["clean_title"] = testdf["clean_title"].apply(stem_words)
testdf[["title","clean_title"]]

,title,clean_title
0,Farm dog,farm dog
1,Mystery of why dog got stuck in wall,mysteri dog get stuck wall
2,Aggressive AKK. How do I fix this?,aggress akk fix
3,An enjoyable mystery about dogs,enjoy mysteri dog
4,"GSD allergic to something, need help figuring out what it is.",gsd allerg someth need help figur
...,...,...
1584,Cat vs HC Helicopter. An Exotic Short Hair Cat battles a Air Hogs Havoc Helicopter. A Classic from last year.,cat vs hc helicopt exot short hair cat battl air hog havoc helicopt classic last year
1585,"Cat From Hell. Funny, or stupid animal abuse?",cat hell funni stupid anim abus
1586,Neelix loves the Hefeweizen,neelix love hefeweizen
1587,Neelix loves the Hefeweizen,neelix love hefeweizen


In [55]:
testdffinal = testdf[['subreddit', 'clean_title']]

In [56]:
testdffinal

,subreddit,clean_title
0,dogs,farm dog
1,dogs,mysteri dog get stuck wall
2,dogs,aggress akk fix
3,dogs,enjoy mysteri dog
4,dogs,gsd allerg someth need help figur
...,...,...
1584,cats,cat vs hc helicopt exot short hair cat battl air hog havoc helicopt classic last year
1585,cats,cat hell funni stupid anim abus
1586,cats,neelix love hefeweizen
1587,cats,neelix love hefeweizen


In [57]:
testdffinal.to_csv('../project_3/CSV files/testdffinal.csv', sep=';', header=True, index=False, columns=[
    'subreddit', 'clean_title'])

In [59]:
testdffinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3050 entries, 0 to 1588
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subreddit    3050 non-null   object
 1   clean_title  3050 non-null   object
dtypes: object(2)
memory usage: 71.5+ KB


In [68]:
testdffinal = pd.read_csv('../project_3/CSV files/testdffinal.csv', sep=';')

In [69]:
testdffinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3050 entries, 0 to 3049
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subreddit    3050 non-null   object
 1   clean_title  3044 non-null   object
dtypes: object(2)
memory usage: 47.8+ KB


In [70]:
testdffinal.isna().sum()

subreddit      0
clean_title    6
dtype: int64

In [71]:
testdffinal.dropna(subset = ["clean_title"], inplace=True)

In [72]:
testdffinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3044 entries, 0 to 3049
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subreddit    3044 non-null   object
 1   clean_title  3044 non-null   object
dtypes: object(2)
memory usage: 71.3+ KB


In [73]:
testdffinal['subreddit'] = testdffinal['subreddit'].map({'dogs': 1, 'cats': 0})

## 5.3 Applying Logreg model to dataframe

In [78]:
X_check = testdffinal['clean_title']
y_check = testdffinal['subreddit']

In [83]:
# Split data into train and test sets
X_check_train, X_check_test, y_check_train, y_check_test = train_test_split(X_check, y_check, 
                                                    test_size=.33, 
                                                    random_state=42)

In [84]:
import pickle

In [85]:
filename = 'finalized_model.sav'

In [86]:
lr_loaded_model = pickle.load(open(filename, 'rb'))

In [88]:
# Transform X_train with TFIDFVectorizer
tvec = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=500)
Z_check_train = tvec.fit_transform(X_check_train)

In [89]:
# fit model
lr_loaded_model.fit(Z_check_train, y_check_train)

LogisticRegression(max_iter=5000)

#### Training Accuracy

In [91]:
# Score model on train data
lr_loaded_model.score(Z_check_train, y_check_train)

0.9396763119176067

#### Testing Accuracy

In [92]:
# Transform X_test with TFIDFVectorizer
Z_check_test = tvec.transform(X_check_test)

In [93]:
# Score model on test data
lr_loaded_model.score(Z_check_test, y_check_test)

0.8925373134328358

- We pulled another ~1500 posts dated (01 Jan 2010 to 31 Dec 2010) each from subreddits "Dogs" and "cats", and created a new dataframe from these posts.
- After data cleaning, we applied the logistic regression with TFIDF Vectorizer to the dataframe.
- The model still performed well, though there may be some overfitting. The model showed relatively high training (about 93.9% after fit.transform X_check_train set) and testing accuracy (about 89.2% after transforming X_check_test set). It appears that there is some overfitting.